# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list) #

/workspace/home
['/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-28-events.csv', '/workspace/home/event_data/2018-11-29-events.csv', '/workspace/home/event_data/2018-11-13-events.csv', '/workspace/home/event_data/2018-11-14-events.csv', '/workspace/home/event_data/2018-11-03-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-05-events.csv', '/workspace/home/event_data/2018-11-26-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-02-events.csv', '/workspace/home/event_data/2018-11-10-events.csv', '/workspace/home/event_data/2018-11-11-events.csv', '/workspace/home/event_data/2018-11-12-events.csv', '/workspace/home/event_data/2018-11-22-events.csv', '/workspace/home/event_data/2018-11-18-events.csv', '/workspace/home/event_data/2018-11-08-events.csv', '/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-25-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print('\n',full_data_rows_list[0],'\n', full_data_rows_list[1])

##########################
# creating a smaller event data csv file called event_datafile_full csv 
# that will be used to insert data into the Apache Cassandra tables
#########################
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056

 ['Kenny G', 'Logged In', 'Chloe', 'F', '53', 'Cuevas', '256.57424', 'paid', 'San Francisco-Oakland-Hayward, CA', 'PUT', 'NextSong', '1.54094E+12', '648', 'Everlasting', '200', '1.54241E+12', '49'] 
 ['Randy Crawford', 'Logged In', 'Chloe', 'F', '54', 'Cuevas', '251.402', 'paid', 'San Francisco-Oakland-Hayward, CA', 'PUT', 'NextSong', '1.54094E+12', '648', 'Rio De Janeiro Blue (Album Version)', '200', '1.54241E+12', '49']


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
#==================
# Creating a Cluster
#==================

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

except Exception as e:
    print(e)


In [8]:
#Test the connection. Expecting error. 

try:
    session.execute("""select * from music_library """)
except Exception as e:
    print(e)
    

Error from server: code=2200 [Invalid query] message="unconfigured table music_library"


#### Create Keyspace

In [9]:
#===============================================================================================================
# Create a Keyspace named p1_udacity with the SimpleStrategy replication strategy and a replication factor of 1
#===============================================================================================================
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS p1_udacity
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor': 1}"""
                   )
except Exception as e:
	print (e)


#### Set Keyspace

In [10]:
#=====================================================
#Set the keyspace p1_udacity for the Cassandra session
#=====================================================
try:
    session.set_keyspace('p1_udacity')
except Exception as e:
    print(e)


### Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
# CSV file will be used in 3 queries
file = 'event_datafile_new.csv'


##### For building the ETL Pipeline to address each of these questions above, there are concrete 3 steps that we would follow: 
+ Creating each table for each question using CREATE statement 
+ Loading the data into the newly created table using the INSERT INTO statement
+ Validate the data using SELECT FROM WHERE clause to retrieve the specific columns from a table with relevant conditions


##### QUERY 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
For this query we pick the *table song_features*, 
+ Collumn **itemInSession** is used as a partition key because the queries will filter by this column. 
+ Collumn **sessionId** is used as clustering column to help make up a unique key.

*The combination of these two columns will be used to uniquely identify rows in the table.*

In [12]:
#################
# Create table:
#################

session.execute("DROP TABLE IF EXISTS song_features")

# create table for Query 1
song_features = """CREATE TABLE IF NOT EXISTS song_features (
                                itemInSession int, 
                                sessionId int, 
                                artist text, 
                                song text,
                                length float, 
                                PRIMARY KEY(itemInSession, sessionId) 
                                )"""


# execute the newly created table 
session.execute(song_features)


In [13]:
#################
# Insert the data
#################

# insert data for song_features
insert_data_song_features = """INSERT INTO song_features (
                                         itemInSession, 
                                         sessionId, 
                                         artist, 
                                         song, 
                                         length) 
                                VALUES (%s, %s, %s, %s, %s)"""


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # adding columns from csv in each variable
        itemInSession, sessionId, song, artist, length = int(line[3]), int(line[8]), str(line[9]), str(line[0]), float(line[5])
        
        # execute the insertion
        session.execute (insert_data_song_features, (itemInSession, sessionId, song, artist, length))

In [14]:
####################
# Validate the data
###################

query_1 = """SELECT artist, song, length 
                    FROM song_features 
                    WHERE itemInSession = %s AND sessionId = %s"""

rows = session.execute(query_1, (4, 338)) #execute the query according to the question

#print the value of them for each column artist, song, length
for row in rows:
    print(f'artist: {row.artist}, song: {row.song} length: {row.length:.8}')        

artist: Faithless, song: Music Matters (Mark Knight Dub) length: 495.30731


##### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 

For this query we pick the *artist_song_by_user*, 
+ the **userId** and **sessionId** are used as a composite partition key because the queries will filter by these columns.
+ The **itemInSession** are used as clustering column to help make up a unique key.

*The combination of these two columns will be used to uniquely identify rows in the table.*

In [15]:
#====================================================
# Create table

#====================================================

# in case rerun this cell, drop the current table 
session.execute("DROP TABLE IF EXISTS artist_song_by_user")

# create table for Query 1
artist_song_by_user = """CREATE TABLE IF NOT EXISTS artist_song_by_user (
                                                        userId int, 
                                                        sessionId int, 
                                                        itemInSession int, 
                                                        artist text, 
                                                        song text, 
                                                        firstName text, 
                                                        lastName text, 
                                                        PRIMARY KEY ((userId, sessionId), itemInSession)
                                                        )"""
# execute the create table
session.execute(artist_song_by_user)




In [16]:
#================== 
# Insert the data
#==================

insert_data_artist_song_by_user = """INSERT INTO artist_song_by_user (
                                                 userId, 
                                                 sessionId, 
                                                 itemInSession, 
                                                 artist, 
                                                 song, 
                                                 firstName, 
                                                 lastName) 
                                     VALUES (%s, %s, %s, %s, %s, %s, %s)"""


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # adding columns from csv in each variable
        userId, sessionId, itemInSession, song, artist, firstName, lastName  = int(line[10]), int(line[8]), int(line[3]), str(line[9]), \
                                                                                str(line[0]), str(line[1]), str(line[4])
         
        
        # execute the insertion
        session.execute(insert_data_artist_song_by_user, (userId, sessionId, itemInSession, song, artist, firstName, lastName))
        

In [17]:
#==================
# Validate the data
#==================

# Query 2 in CQL
query_2 = """SELECT artist, song, firstName, lastName 
                    FROM artist_song_by_user
                    WHERE userId = %s AND sessionId = %s"""

rows = session.execute(query_2, (10, 182))
for row in rows:
    print(f'artist: {row.artist}, song: {row.song}, user first name: {row.firstname}, user last name: {row.lastname}')

artist: Down To The Bone, song: Keep On Keepin' On, user first name: Sylvie, user last name: Cruz
artist: Three Drives, song: Greece 2000, user first name: Sylvie, user last name: Cruz
artist: Sebastien Tellier, song: Kilometer, user first name: Sylvie, user last name: Cruz
artist: Lonnie Gordon, song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), user first name: Sylvie, user last name: Cruz


##### QUERY 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


For table *user_name*, 
+ the **song** was used as a partition key because the queries will filter by 
this column. 
+ The **userId** were used as clustering column to help make up a unique key.

*The combination of these two columns will be used to uniquely identify rows in the table.*

In [18]:
#====================================================
# Create table

#====================================================

# in case rerun this cell, drop the current table 
session.execute("DROP TABLE IF EXISTS table_query_3")

# create table for Query 1
user_name = """CREATE TABLE IF NOT EXISTS user_name (
                                        song text, 
                                        userId int, 
                                        firstName text, 
                                        lastName text, 
                                        PRIMARY KEY (song, userId)
                                        )"""
# execute the create table
session.execute(user_name)



In [19]:
#==================
# Insert the data
#==================

insert_data_user_name = """INSERT INTO user_name (
                                            song, 
                                            userId, 
                                            firstName, 
                                            lastName
                                            ) 
                                            VALUES (%s, %s, %s, %s)"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # adding columns from csv in each variable
        song, userId, firstName, lastName = str(line[9]), int(line[10]), str(line[1]), str(line[4])
        
        # execute the insertion
        session.execute(insert_data_user_name, (song, userId, firstName, lastName))

In [20]:
#==================
# Verify the data
#==================

# Query 3 in CQL
query_3 = """SELECT firstName, lastName 
                    FROM user_name 
                    WHERE song = %s"""
                    
#the name of the song now used as the query condition
rows = session.execute(query_3, ('All Hands Against His Own', ))

# print the user
for row in rows:
    print(f'user first name: {row.firstname:>10},  user last name: {row.lastname}')    

user first name: Jacqueline,  user last name: Lynch
user first name:      Tegan,  user last name: Levine
user first name:       Sara,  user last name: Johnson


### Drop the tables before closing out the sessions

In [21]:
#=================================================
# Drop the table before closing out the sessions
#=================================================

session.execute("DROP TABLE IF EXISTS song_features")
session.execute("DROP TABLE IF EXISTS artist_song_by_user")
session.execute("DROP TABLE IF EXISTS user_name")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()

## FINISH